### Form
- Form의 역할
    - 데이터를 담아 전송과 처리를 해줌
    - form 덕분에 필수적인 데이터를 놓치지 않고, 필요없는 데이터는 정제할수 있음

### Serializer
- REST Framework에서 form과 동일한 역할을 함
- 차이점
    - REST: HTML form 형식 사용
    - Serializer: json 형식
- 데이터를 담아서 정리해주는 상자의 역할 진행

### CRUD - Read 구현

#### 1. blog app 만들기
1. $ python manage.py startapp blog
2. sttings.py의 INSTALLED_APPS에 등록

#### 2. models.py 작성
- 모델 수정이후 반드시 마이그레이션 해주기!


$ python manage.py makemigrations

$ python manage.py migrate


- 모델 등록후 admin 페이지에 알려주기

In [ ]:
# models.py

from django.db import models # 아래에서 사용

class Blog(models.Model):
    title = models.CharField(max_length=100) # 길이제한
    body = models.TextField(default="default")
    date = models.DateTimeField(auto_now_add=True) # 생성일자에 사용
    # auto-now는 수정일자 / auto-now-add는 생성일자

    def __str__(self):
        return self.title

In [ ]:
# admin.py

from django.contrib import admin
from .models import *

admin.site.register(Blog) # model을 등록해줬기 때문에 admin애도 알리기

#### 3. urls.py 작성
- path에 추가해주기
- import 부분도 추가시 신경써야함

In [ ]:
# tutorial -> urls.py

from django.contrib import admin
from django.urls import path, include # include 추가

urlpatterns = [
    path('admin/', admin.site.urls),
    path('blog/', include('blog.urls')), # blog.urls를 포함
]

In [ ]:
# blog -> urls.py

from django.urls import path
from .views import *

app_name = 'blog'

urlpatterns = [
    path('', get_all_blogs),
    path('<int:pk>/', get_one_blog)
]

#### 4. serializer 생성하기
- blog 앱 안에 serializers.py 생성하기

In [ ]:
# blog -> serializers.py

from rest_framework import serializers # serializer 사용
from dataclasses import field
from .models import Blog

class BlogSerializer(serializers.ModelSerializer): # ModelSerializer를 상속
    class Meta: # Meta 클래스 사용
        model = Blog
        fields = ['id', 'title', 'body']
        read_only_fields = ['id'] # 읽는것만 가능한 필드 -> id에 대한 조작을 방지

### Meta 클래스
- Meta 클래스란?
    - model 클래스에 대한 정의를 해주는 부분 -> model에 대한 다양한 정의가 가능
    - 내부 클래스로 사용되어 상위 클래스에게 meta data를 제공
    - 데이터에 대한 데이터 or 다른 데이터를 설명해주는 데이터
    - ex) 데이터베이스 이름, 단 복수 이름, 추상화, 순서 지정

#### 5. views.py 작성하기

In [ ]:
from .models import Blog
from .serializers import BlogSerializer # 데이터를 담은 serializer
from rest_framework.decorators import api_view # http method 지정 데코레이터
from rest_framework.response import Response # 응답
from rest_framework import status # for http status code

'''
전체 블로그를 조회
'''
@api_view(['GET']) # GET 요청만 받겠다.
def get_all_blogs(request):
    blogs = Blog.objects.all()
    serializer = BlogSerializer(blogs, many=True) # 'many=True' -> 다수의 blogs 형태를 serialize하고자 할 때
    # serialize로 보내려는 객체가 복수일 때 many=True 반드시 해야함
    return Response(serializer.data)

'''
한 블로그 조회
'''
@api_view(['GET'])
def get_one_blog(request, pk):
    try: # 블로그가 존재할 경우
        blog = Blog.objects.get(pk=pk)
        serializer = BlogSerializer(blog)
        return Response(serializer.data)
    except Blog.DoesNotExist: # 블로그 존재하지 않을 경우
        return Response(status=status.HTTP_404_NOT_FOUND)

### @api_view 데코레이터

- 함수가 요청 받을 http method를 지정해주는 데코레이터
- 지정된 http method가 아닐 경우 무시
    - ex) GET -> POST
- 두개의 http method를 받을 수 있음
    - ex) ['GET','POST']
- api를 사용하기 위해서는 반드시 사용